In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from evaluate_models import plot_cm, process_labels, calc_precision_recall, calc_f1
from matplotlib import pyplot as plt
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from custom_plots import gen_violin_plot, tensor_to_val
import os

In [2]:
X = np.load('../data/galaxy_X.npy')
y = np.load('../data/galaxy_y.npy')
y_aux = np.load('../data/galaxy_y_aux.npy')
y_man_aux = np.load('../data/galaxy_y_manual_aux.npy')

In [3]:
#Set seeds to be able to reproduce results
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
X_tmp_train, X_test1, y_aux_tmp_train, y_aux_test1, y_man_aux_tmp_train, y_man_aux_test1, y_tmp_train, y_test = train_test_split(X, y_aux, y_man_aux, y, test_size=0.1, train_size=0.9, random_state=42, shuffle=True, stratify=y)
X_train1, X_val1, y_aux_train1, y_aux_val1, y_man_aux_train1, y_man_aux_val1, y_train, y_val = train_test_split(X_tmp_train, y_aux_tmp_train, y_man_aux_tmp_train, y_tmp_train, test_size=0.11, train_size=0.89, random_state=42, shuffle=True, stratify=y_tmp_train)

In [5]:
def normalize_images(X):
    return np.array(list(map(normalize_image, X)))
    
def normalize_image(img):
    bot = np.min(img)
    top = np.max(img)
    norm = (img - bot)/(top - bot)
    return norm

X_train = normalize_images(X_train1)
X_val = normalize_images(X_val1)
X_test = normalize_images(X_test1)

X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [6]:
def normalize_targets(y):
    bot = np.min(y, axis=0)
    top = np.max(y, axis=0)
    norm = (y - bot)/(top - bot)
    return norm

y_aux_train = normalize_targets(y_aux_train1)
y_aux_val = normalize_targets(y_aux_val1)
y_aux_test = normalize_targets(y_aux_test1)
y_man_aux_train = normalize_targets(y_man_aux_train1)
y_man_aux_val = normalize_targets(y_man_aux_val1)
y_man_aux_test = normalize_targets(y_man_aux_test1)

In [7]:
def extract_class_pred(preds):
    return preds[0]

In [8]:
def reshape_predictions(pred):
    length = len(pred[0])
    new_preds = []
    for j in range(length):
        feats = []
        for i in range(4):
            feats.append(pred[i][j, 0])
        new_preds.append(feats)
    return np.array(new_preds)

In [9]:
y_test_bent = y_aux_test[:, 0]
y_test_fr = y_aux_test[:, 1]
y_test_cores = y_aux_test[:, 2]
y_test_size = y_aux_test[:, 3]
test_labels = {
    'main_out': y_test,
    'bent_out': y_test_bent,
    'fr_out': y_test_fr,
    'cores_out': y_test_cores,
    'size_out': y_test_size
}

In [10]:
y_man_test_bent = y_man_aux_test[:, 0]
y_man_test_fr = y_man_aux_test[:, 1]
y_man_test_cores = y_man_aux_test[:, 2]
y_man_test_size = y_man_aux_test[:, 3]
man_test_labels = {
    'main_out': y_test,
    'bent_out': y_man_test_bent,
    'fr_out': y_man_test_fr,
    'cores_out': y_man_test_cores,
    'size_out': y_man_test_size
}

In [11]:
bent_f1s = []
comp_f1s = []
fri_f1s = []
frii_f1s = []
macro_f1s = []
model = []
run_log = []
epochs = []
accuracy = []
full_loss = []
main_loss = []
overfit = []
overfit_scores = []
true_labels = []

In [12]:
# Determine even merged performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):        
    loc = f'../models/merged_0.5_0.5_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Even\nMCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/merged_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    # Use validation data to calculate overfitting
    name = f'../lr_logs/merged_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/merged_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/merged_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [13]:
# Determine aux merged performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/merged_0.25_0.75_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Aux\nMCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/merged_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/merged_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/merged_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/merged_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [14]:
# Determine main merged performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/merged_0.75_0.25_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Main\nMCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/merged_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/merged_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/merged_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/merged_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [15]:
# Determine even multihead performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/multihead_0.5_0.5_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Even\nMhCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/multihead_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/multihead_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/multihead_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/multihead_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [16]:
# Determine aux multihead performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/multihead_0.25_0.75_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Aux\nMhCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/multihead_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/multihead_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/multihead_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/multihead_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [17]:
# Determine main multihead performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/multihead_0.75_0.25_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Main\nMhCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/multihead_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/multihead_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/multihead_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/multihead_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [18]:
# Determine standard performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    best_model = keras.models.load_model(f"../models/standard_model{run}.h5")
    tmp_loss, tmp_acc = best_model.evaluate(X_test, y_test, verbose=0)
    test_pred = best_model.predict(X_test)
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('SCNN')
    accuracy.append(tmp_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_loss)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/standard_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/standard_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [19]:
# Determine wide performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    aux_model = keras.models.load_model(f'../models/aux_model{run}.h5')
    X_test_pred = aux_model.predict(X_test)
    X_test_aux = reshape_predictions(X_test_pred) 
    best_model = keras.models.load_model(f"../models/wide_model{run}.h5")
    tmp_loss, tmp_acc = best_model.evaluate((X_test_aux, X_test), y_test, verbose=0)
    test_pred = best_model.predict((X_test_aux, X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('WCNN')
    accuracy.append(tmp_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_loss)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/wide_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/wide_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Automated')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/wide_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/wide_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [20]:
# Determine even merged performance with additional manual labels
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):        
    loc = f'../models/man_merged_0.5_0.5_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, man_test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Even\nMCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_merged_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/man_merged_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_merged_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/man_merged_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [21]:
# Determine aux merged performance with additional manual labels
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/man_merged_0.25_0.75_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, man_test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Aux\nMCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_merged_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/man_merged_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_merged_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/man_merged_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [22]:
# Determine main merged performance with additional manual labels
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/man_merged_0.75_0.25_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, man_test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Main\nMCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_merged_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/man_merged_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_merged_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/man_merged_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [23]:
# Determine even multihead performance with additional manual labels
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/man_multihead_0.5_0.5_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, man_test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Even\nMhCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_multihead_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/man_multihead_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_multihead_0.5_0.5_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/man_multihead_0.5_0.5_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [24]:
# Determine auxiliary multihead performance with additional manual labels
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/man_multihead_0.25_0.75_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, man_test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Aux\nMhCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_multihead_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/man_multihead_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_multihead_0.25_0.75_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/man_multihead_0.25_0.75_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [25]:
# Determine main multihead performance with additional manual labels
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    loc = f'../models/man_multihead_0.75_0.25_model{run}.h5'
    best_model = keras.models.load_model(loc)
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = best_model.evaluate(X_test, man_test_labels, verbose=0)
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Main\nMhCNN')
    accuracy.append(tmp_lbl_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_lbl_entropy)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_multihead_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/man_multihead_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_multihead_0.75_0.25_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/man_multihead_0.75_0.25_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [26]:
# Determine standard performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    best_model = keras.models.load_model(f"../models/standard_model{run}.h5")
    tmp_loss, tmp_acc = best_model.evaluate(X_test, y_test, verbose=0)
    test_pred = best_model.predict(X_test)
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('SCNN')
    accuracy.append(tmp_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_loss)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/standard_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/standard_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [27]:
# Determine wide performance with additional manual labels
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    aux_model = keras.models.load_model(f'../models/man_aux_model{run}.h5')
    X_test_pred = aux_model.predict(X_test)
    X_test_aux = reshape_predictions(X_test_pred) 
    best_model = keras.models.load_model(f"../models/man_wide_model{run}.h5")
    tmp_loss, tmp_acc = best_model.evaluate((X_test_aux, X_test), y_test, verbose=0)
    test_pred = best_model.predict((X_test_aux, X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('WCNN')
    accuracy.append(tmp_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_loss)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_wide_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/man_wide_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    true_labels.append('Manual')
    
    all_losses = all_losses + train_loss + val_loss

# Fit scaler to normalize range of loss values
all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

# Second loop through data to normalize loss values and calculate overfitting score
for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/man_wide_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/man_wide_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [28]:
data = {
    'Model': model,
    'Macro F1': macro_f1s,
    'Bent F1': bent_f1s,
    'Comp F1': comp_f1s,
    'FRI F1': fri_f1s,
    'FRII F1': frii_f1s,
    'Accuracy': accuracy,
    'Aggregated Loss': full_loss,
    'Classification Loss': main_loss,
    'Epochs': epochs,
    'Overfitting ratio': overfit,
    'Overfitting score': overfit_scores,
    'Feature Vector Type': true_labels,
    'Run': run_log
}
df = pd.DataFrame(data)

In [30]:
if not os.path.exists("../plots/Galaxy/"):
    if os.getcwd().split("/")[-1] == "src":
        os.mkdir("../plots/Galaxy")
    else:
        print("This function should only be called from the src directory")
gen_violin_plot(df, 'Model', 'Macro F1', 'Feature Vector Type', 'Galaxy/DualMacroF1s.eps')
gen_violin_plot(df, 'Model', 'Bent F1', 'Feature Vector Type', 'Galaxy/DualBentF1s.eps')
gen_violin_plot(df, 'Model', 'Comp F1', 'Feature Vector Type', 'Galaxy/DualCompF1s.eps')
gen_violin_plot(df, 'Model', 'FRI F1', 'Feature Vector Type', 'Galaxy/DualFRIF1s.eps')
gen_violin_plot(df, 'Model', 'FRII F1', 'Feature Vector Type', 'Galaxy/DualFRIIF1s.eps')
gen_violin_plot(df, 'Model', 'Accuracy', 'Feature Vector Type', 'Galaxy/DualAccuracies.eps')
gen_violin_plot(df, 'Model', 'Aggregated Loss', 'Feature Vector Type', 'Galaxy/DualFullLosses.eps')
gen_violin_plot(df, 'Model', 'Classification Loss', 'Feature Vector Type', 'Galaxy/DualClassLoss.eps')
gen_violin_plot(df, 'Model', 'Epochs', 'Feature Vector Type', 'Galaxy/DualEpochsFull.eps')
gen_violin_plot(df, 'Model', 'Epochs', 'Feature Vector Type', 'Galaxy/DualEpochs.eps', only_data=True, limit=True, limit_range=[10, 28])
gen_violin_plot(df, 'Model', 'Overfitting ratio', 'Feature Vector Type', 'Galaxy/DualOverfitFull.eps')
gen_violin_plot(df, 'Model', 'Overfitting ratio', 'Feature Vector Type', 'Galaxy/DualOverfit.eps', only_data=True)
gen_violin_plot(df, 'Model', 'Overfitting score', 'Feature Vector Type', 'Galaxy/DualOverfitScoreFull.eps')
gen_violin_plot(df, 'Model', 'Overfitting score', 'Feature Vector Type', 'Galaxy/DualOverfitScore.eps', only_data=True, limit=True, limit_range=[0, 1])

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [31]:
df2 = df[df['Feature Vector Type'] == 'Automated']
gen_violin_plot(df2, 'Model', 'Macro F1', 'Feature Vector Type', 'Galaxy/FullMacroF1s.eps', split=False)
gen_violin_plot(df2, 'Model', 'Macro F1', 'Feature Vector Type', 'Galaxy/MacroF1s.eps', split=False, only_data=True)
gen_violin_plot(df2, 'Model', 'Bent F1', 'Feature Vector Type', 'Galaxy/BentF1s.eps', split=False)
gen_violin_plot(df2, 'Model', 'Comp F1', 'Feature Vector Type', 'Galaxy/CompF1s.eps', split=False)
gen_violin_plot(df2, 'Model', 'FRI F1', 'Feature Vector Type', 'Galaxy/FRIF1s.eps', split=False)
gen_violin_plot(df2, 'Model', 'FRII F1', 'Feature Vector Type', 'Galaxy/FRIIF1s.eps', split=False)
gen_violin_plot(df2, 'Model', 'Accuracy', 'Feature Vector Type', 'Galaxy/FullAccuracies.eps', split=False)
gen_violin_plot(df2, 'Model', 'Accuracy', 'Feature Vector Type', 'Galaxy/Accuracies.eps', split=False, only_data=True)
gen_violin_plot(df2, 'Model', 'Aggregated Loss', 'Feature Vector Type', 'Galaxy/FullLosses.eps', split=False)
gen_violin_plot(df2, 'Model', 'Classification Loss', 'Feature Vector Type', 'Galaxy/FullClassLoss.eps', split=False)
gen_violin_plot(df2, 'Model', 'Classification Loss', 'Feature Vector Type', 'Galaxy/ClassLoss.eps', split=False, only_data=True)
gen_violin_plot(df2, 'Model', 'Epochs', 'Feature Vector Type', 'Galaxy/FullEpochs.eps', split=False)
gen_violin_plot(df2, 'Model', 'Epochs', 'Feature Vector Type', 'Galaxy/Epochs.eps', split=False, only_data=True)
gen_violin_plot(df2, 'Model', 'Overfitting ratio', 'Feature Vector Type', 'Galaxy/FullOverfit.eps', split=False)
gen_violin_plot(df2, 'Model', 'Overfitting ratio', 'Feature Vector Type', 'Galaxy/Overfit.eps', only_data=True, split=False)
gen_violin_plot(df2, 'Model', 'Overfitting score', 'Feature Vector Type', 'Galaxy/FullOverfitScore.eps', split=False)
gen_violin_plot(df2, 'Model', 'Overfitting score', 'Feature Vector Type', 'Galaxy/OverfitScore.eps', split=False, only_data=True)

In [32]:
# Save dataframes for future use
if not os.path.exists('../Dataframes/'):
        os.makedirs('../Dataframes/')
df.to_csv('../Dataframes/combined.csv', index=False)
df2.to_csv('../Dataframes/automatic.csv', index=False)

# Derotated comparison

In [12]:
X_test1 = np.load('../data/galaxy_X_test1.npy')
y_test = np.load('../data/galaxy_y_test.npy')
X_der_test1 = np.load('../data/galaxy_X_test_derotated.npy')

In [13]:
#Set seeds to be able to reproduce results
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
def normalize_images(X):
    return np.array(list(map(normalize_image, X)))
    
def normalize_image(img):
    bot = np.min(img)
    top = np.max(img)
    norm = (img - bot)/(top - bot)
    return norm

X_test = normalize_images(X_test1)
X_test = X_test[..., np.newaxis]

X_der_test = normalize_images(X_der_test1)
X_der_test = X_der_test[..., np.newaxis]

In [15]:
bent_f1s = []
comp_f1s = []
fri_f1s = []
frii_f1s = []
macro_f1s = []
model = []
run_log = []
epochs = []
accuracy = []
full_loss = []
main_loss = []
overfit = []
overfit_scores = []
train_times = []

In [16]:
std_times = np.load('../time_logs/standard_times.npy')
std_aug_times = np.load('../time_logs/standard_aug_times.npy')
std_derotate_times = np.load('../time_logs/std_derotate_times.npy')

In [17]:
# Determine standard performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    train_times.append(std_times[run-1])
    
    best_model = keras.models.load_model(f"../models/standard_model{run}.h5")
    tmp_loss, tmp_acc = best_model.evaluate(X_test, y_test, verbose=0)
    test_pred = best_model.predict(X_test)
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Standard')
    accuracy.append(tmp_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_loss)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/standard_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    
    all_losses = all_losses + train_loss + val_loss

all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/standard_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [18]:
# Determine standard performance with standardized rotation
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    train_times.append(std_derotate_times[run-1])
    
    best_model = keras.models.load_model(f"../models/derotated_standard_model{run}.h5")
    tmp_loss, tmp_acc = best_model.evaluate(X_der_test, y_test, verbose=0)
    test_pred = best_model.predict(X_der_test)
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Derotated')
    accuracy.append(tmp_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_loss)
    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_derotated_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/standard_derotated_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    
    all_losses = all_losses + train_loss + val_loss

all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_derotated_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/standard_derotated_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [19]:
# Determine standard augmented performance
runs = 20
all_losses = []
scaler = MinMaxScaler()
for run in range(1,runs+1):
    train_times.append(std_aug_times[run-1])
    
    best_model = keras.models.load_model(f"../models/aug_standard_model{run}.h5")
    tmp_loss, tmp_acc = best_model.evaluate(X_test, y_test, verbose=0)
    test_pred = best_model.predict(X_test)
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)
    run_log.append(run)
    model.append('Augmented')
    accuracy.append(tmp_acc)
    full_loss.append(tmp_loss)
    main_loss.append(tmp_loss)
    
    # Get loss, accuracy and epoch information 
    name = f'../lr_logs/standard_aug_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    epochs.append(epoch_loss[-1] + 1)
    
    name = f'../lr_logs/standard_aug_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    overfit.append(val_loss[-1]/train_loss[-1])
    
    all_losses = all_losses + train_loss + val_loss

all_losses = np.array(all_losses).reshape(-1, 1)
scaler.fit(all_losses)

for run in range(1,runs+1):    
    # Get loss, accuracy and epoch information    
    name = f'../lr_logs/standard_aug_run{run}/train'
    loss_name = 'epoch_loss'
    ea = EventAccumulator(name)
    ea.Reload()
    ct, epoch_loss, train_loss = zip(*ea.Tensors(loss_name))
    epoch_loss = list(epoch_loss)
    train_loss = list(train_loss)
    train_loss= list(map(tensor_to_val, train_loss))
    train_loss = np.array(train_loss).reshape(-1, 1)
    train_loss = scaler.transform(train_loss)
    
    name = f'../lr_logs/standard_aug_run{run}/validation'
    ea = EventAccumulator(name)
    ea.Reload()
    _, _, val_loss = zip(*ea.Tensors(loss_name))
    val_loss = list(val_loss)
    val_loss= list(map(tensor_to_val, val_loss))
    val_loss = np.array(val_loss).reshape(-1, 1)
    val_loss = scaler.transform(val_loss)
    
    overfit_scores.append(val_loss[-1, 0] - train_loss[-1, 0])

In [21]:
data = {
    'Model': model,
    'Macro F1': macro_f1s,
    'Bent F1': bent_f1s,
    'Comp F1': comp_f1s,
    'FRI F1': fri_f1s,
    'FRII F1': frii_f1s,
    'Accuracy': accuracy,
    'Classification Loss': main_loss,
    'Epochs': epochs,
    'Overfitting ratio': overfit,
    'Overfitting scores': overfit_scores,
    'Training times': train_times,
    'Run': run_log
}
df = pd.DataFrame(data)

if not os.path.exists("../plots/Galaxy/"):
    if os.getcwd().split("/")[-1] == "src":
        os.mkdir("../plots/Galaxy")
    else:
        print("This function should only be called from the src directory")

gen_violin_plot(df, 'Model', 'Macro F1', 'True feature vectors', 'Galaxy/RotationFullMacroF1s.eps', split=False)
gen_violin_plot(df, 'Model', 'Macro F1', 'True feature vectors', 'Galaxy/RotationMacroF1s.eps', split=False, only_data=True)
gen_violin_plot(df, 'Model', 'Bent F1', 'True feature vectors', 'Galaxy/RotationBentF1s.eps', split=False)
gen_violin_plot(df, 'Model', 'Comp F1', 'True feature vectors', 'Galaxy/RotationCompF1s.eps', split=False)
gen_violin_plot(df, 'Model', 'FRI F1', 'True feature vectors', 'Galaxy/RotationFRIF1s.eps', split=False)
gen_violin_plot(df, 'Model', 'FRII F1', 'True feature vectors', 'Galaxy/RotationFRIIF1s.eps', split=False)
gen_violin_plot(df, 'Model', 'Accuracy', 'True feature vectors', 'Galaxy/RotationFullAccuracies.eps', split=False)
gen_violin_plot(df, 'Model', 'Accuracy', 'True feature vectors', 'Galaxy/RotationAccuracies.eps', split=False, only_data=True)
gen_violin_plot(df, 'Model', 'Classification Loss', 'True feature vectors', 'Galaxy/RotationFullClassLoss.eps', split=False)
gen_violin_plot(df, 'Model', 'Classification Loss', 'True feature vectors', 'Galaxy/RotationClassLoss.eps', split=False, only_data=True)
gen_violin_plot(df, 'Model', 'Epochs', 'True feature vectors', 'Galaxy/RotationFullEpochs.eps', split=False)
gen_violin_plot(df, 'Model', 'Epochs', 'True feature vectors', 'Galaxy/RotationEpochs.eps', split=False, only_data=True)
gen_violin_plot(df, 'Model', 'Overfitting ratio', 'True feature vectors', 'Galaxy/RotationOverfitFull.eps', split=False)
gen_violin_plot(df, 'Model', 'Overfitting ratio', 'True feature vectors', 'Galaxy/RotationOverfit.eps', split=False, only_data=True)
gen_violin_plot(df, 'Model', 'Training times', 'True feature vectors', 'Galaxy/RotationFullTimes.eps', split=False)
gen_violin_plot(df, 'Model', 'Training times', 'True feature vectors', 'Galaxy/RotationTimes.eps', split=False, only_data=True)
gen_violin_plot(df, 'Model', 'Overfitting scores', 'True feature vectors', 'Galaxy/RotationFullOverfitScores.eps', split=False)
gen_violin_plot(df, 'Model', 'Overfitting scores', 'True feature vectors', 'Galaxy/RotationOverfitScores.eps', split=False, only_data=True)